# Merge, join and concatenate

This is daru translated version of http://pandas.pydata.org/pandas-docs/stable/merging.html

In [1]:
require 'daru'

true

## Concatenating objects

In [2]:
df1 = Daru::DataFrame.new(
  {
    A:['A0', 'A1', 'A2', 'A3'],
    B:['B0', 'B1', 'B2', 'B3'],
    C:['C0', 'C1', 'C2', 'C3'],
    D:['D0', 'D1', 'D2', 'D3']
  }
)

df2 = Daru::DataFrame.new(
  {
    A:['A4', 'A5', 'A6', 'A7'],
    B:['B4', 'B5', 'B6', 'B7'],
    C:['C4', 'C5', 'C6', 'C7'],
    D:['D4', 'D5', 'D6', 'D7']
  }
)

df3 = Daru::DataFrame.new(
  {
    A:['A8', 'A9', 'A10', 'A11'],
    B:['B8', 'B9', 'B10', 'B11'],
    C:['C8', 'C9', 'C10', 'C11'],
    D:['D8', 'D9', 'D10', 'D11']
  }
)
df1.concat(df2).concat(df3)

#<Daru::DataFrame(12x4)>
       A   B   C   D
   0  A0  B0  C0  D0
   1  A1  B1  C1  D1
   2  A2  B2  C2  D2
   3  A3  B3  C3  D3
   4  A4  B4  C4  D4
   5  A5  B5  C5  D5
   6  A6  B6  C6  D6
   7  A7  B7  C7  D7
   8  A8  B8  C8  D8
   9  A9  B9  C9  D9
  10 A10 B10 C10 D10
  11 A11 B11 C11 D11

In [3]:
# can't generate new multi-index
# pd.concate([df1, df2], keys=['X', 'Y'])

In [4]:
# concatenation doesn't keep multi-index
# index become [0,1,2,3...]

### Set logic on the other axes

In [5]:
df4 = Daru::DataFrame.new(
  {
    B: ['B2', 'B3', 'B6', 'B7'],
    D: ['D2', 'D3', 'D6', 'D7'],
    F: ['F2', 'F3', 'F6', 'F7']
  },
  index: [2, 3, 6, 7]
)

#<Daru::DataFrame(4x3)>
       B   D   F
   2  B2  D2  F2
   3  B3  D3  F3
   6  B6  D6  F6
   7  B7  D7  F7

#### outer join

In [6]:
df1.join(df4, on: [:B, :D], how: :outer)

#<Daru::DataFrame(6x5)>
       A   C   B   D   F
   0  A0  C0  B0  D0 nil
   1  A1  C1  B1  D1 nil
   2  A2  C2  B2  D2  F2
   3  A3  C3  B3  D3  F3
   4 nil nil  B6  D6  F6
   5 nil nil  B7  D7  F7

#### inner join

In [7]:
df1.join(df4, on: [:B, :D], how: :inner)

#<Daru::DataFrame(2x5)>
       A   C   B   D   F
   0  A2  C2  B2  D2  F2
   1  A3  C3  B3  D3  F3

### Concatenating using append

In [8]:
df1.concat(df2).concat(df3)

#<Daru::DataFrame(12x4)>
       A   B   C   D
   0  A0  B0  C0  D0
   1  A1  B1  C1  D1
   2  A2  B2  C2  D2
   3  A3  B3  C3  D3
   4  A4  B4  C4  D4
   5  A5  B5  C5  D5
   6  A6  B6  C6  D6
   7  A7  B7  C7  D7
   8  A8  B8  C8  D8
   9  A9  B9  C9  D9
  10 A10 B10 C10 D10
  11 A11 B11 C11 D11

### Ignoring indexes on the concatenatin axis

Ignoring indexes is default behavior of `DataFrame#concat` method.

### Concatenating with mixed ndims

You can concatenate DataFrames with Vectors through `Vector#to_df` method.

In [9]:
v1 = Daru::Vector.new(['V1', 'V2', 'V3', 'V4'], name: :X)
Daru::DataFrame.new(df1.to_hash.merge(v1.to_df.to_hash))

#<Daru::DataFrame(4x5)>
       A   B   C   D   X
   0  A0  B0  C0  D0  V1
   1  A1  B1  C1  D1  V2
   2  A2  B2  C2  D2  V3
   3  A3  B3  C3  D3  V4

### More concatenating with group keys

In [10]:
v2 = Daru::Vector.new([:a, :v, :c, :d], name:'foo')
v3 = Daru::Vector.new([:x, :y, :z, :z])
v4 = Daru::Vector.new([:e, :f, :g, :h])
# when concatenate several dataframes without name like below, the last one is kept and others disappear
df5 = Daru::DataFrame.new(v2.to_df.to_hash.merge(v3.to_df.to_hash).merge(v4.to_df.to_hash))

#<Daru::DataFrame(4x2)>
         foo
   0   e   a
   1   f   v
   2   g   c
   3   h   d

### Appending rows to a DataFrame

In [54]:
df1.concat(
  Daru::DataFrame.rows([['X0', 'X1', 'X2', 'X3']], order: [:A, :B, :C, :D])
)

#<Daru::DataFrame(5x4)>
       A   B   C   D
   0  A0  B0  C0  D0
   1  A1  B1  C1  D1
   2  A2  B2  C2  D2
   3  A3  B3  C3  D3
   4  X0  X1  X2  X3

In [41]:
_df1 = df1.dup
# "add_row" method makes side-effect, not returning new DataFrame
_df1.add_row(['X0', 'X1', 'X2', 'X3'])
_df1

#<Daru::DataFrame(5x4)>
       A   B   C   D
   0  A0  B0  C0  D0
   1  A1  B1  C1  D1
   2  A2  B2  C2  D2
   3  A3  B3  C3  D3
   4  X0  X1  X2  X3

In [53]:
df1.concat(
  Daru::DataFrame.rows([['X0', 'X1', 'X2', 'X3']], order: [:A, :B, :C, :X])
)

#<Daru::DataFrame(5x5)>
       A   B   C   D   X
   0  A0  B0  C0  D0 nil
   1  A1  B1  C1  D1 nil
   2  A2  B2  C2  D2 nil
   3  A3  B3  C3  D3 nil
   4  X0  X1  X2 nil  X3

## Database-style DataFrame joining/merging

### Brief primer on merge methods (relational algebra)

In [13]:
left = Daru::DataFrame.new(
  key: ['K0', 'K1', 'K2', 'K3'],
  A: ['A0', 'A1', 'A2', 'A3'],
  B: ['B0', 'B1', 'B2', 'B3']
)

right = Daru::DataFrame.new(
  key: ['K0', 'K1', 'K2', 'K3'],
  C: ['C0', 'C1', 'C2', 'C3'],
  D: ['D0', 'D1', 'D2', 'D3']
)

left.join(right, on:[:key], how: :inner)

#<Daru::DataFrame(4x5)>
       A   B key   C   D
   0  A0  B0  K0  C0  D0
   1  A1  B1  K1  C1  D1
   2  A2  B2  K2  C2  D2
   3  A3  B3  K3  C3  D3

In [14]:
left = Daru::DataFrame.new(
  key1: ['K0', 'K0', 'K1', 'K2'],
  key2: ['K0', 'K1', 'K0', 'K1'],
  A: ['A0', 'A1', 'A2', 'A3'],
  B: ['B0', 'B1', 'B2', 'B3']
)

right = Daru::DataFrame.new(
  key1: ['K0', 'K1', 'K1', 'K2'],
  key2: ['K0', 'K0', 'K0', 'K0'],
  C: ['C0', 'C1', 'C2', 'C3'],
  D: ['D0', 'D1', 'D2', 'D3']
)

left.join(right, on:[:key1, :key2], how: :inner)

#<Daru::DataFrame(3x6)>
         A    B key1 key2    C    D
    0   A0   B0   K0   K0   C0   D0
    1   A2   B2   K1   K0   C1   D1
    2   A2   B2   K1   K0   C2   D2

In [15]:
left.join(right, on:[:key1, :key2], how: :left)

#<Daru::DataFrame(5x6)>
         A    B key1 key2    C    D
    0   A0   B0   K0   K0   C0   D0
    1   A1   B1   K0   K1  nil  nil
    2   A2   B2   K1   K0   C1   D1
    3   A2   B2   K1   K0   C2   D2
    4   A3   B3   K2   K1  nil  nil

In [16]:
left.join(right, on:[:key1, :key2], how: :outer)

#<Daru::DataFrame(6x6)>
         A    B key1 key2    C    D
    0   A0   B0   K0   K0   C0   D0
    1   A1   B1   K0   K1  nil  nil
    2   A2   B2   K1   K0   C1   D1
    3   A2   B2   K1   K0   C2   D2
    4  nil  nil   K2   K0   C3   D3
    5   A3   B3   K2   K1  nil  nil

In [17]:
left.join(right, on:[:key1, :key2], how: :inner)

#<Daru::DataFrame(3x6)>
         A    B key1 key2    C    D
    0   A0   B0   K0   K0   C0   D0
    1   A2   B2   K1   K0   C1   D1
    2   A2   B2   K1   K0   C2   D2

### The merge indicator

In [18]:
# there's no indicator option.

### Joining on index

In [19]:
# can't join on index

#left = Daru::DataFrame.new({
#  A: ['A0', 'A1', 'A2', 'A3'],
#  B: ['B0', 'B1', 'B2', 'B3']},
#  index: ['K0', 'K1', 'K2', 'K3']
#)
#
#right = Daru::DataFrame.new({
#  C: ['C0', 'C1', 'C2', 'C3'],
#  D: ['D0', 'D1', 'D2', 'D3']},
#  index: ['K0', 'K1', 'K2', 'K3'],
#)
#
#left.join(right, how: :inner)

ArgumentError: Both dataframes expected to have :key field

### Joining key columns on an index

In [20]:
## no feature
# joining between name and index

### Joining a single Index to a Multi-index

In [21]:
## no feature
## multi-index feature also doesn't exist.

### Joining with two multi-indexes

In [22]:
## no feature
## multi-index feature also doesn't exist.

### Overlapping value columns

In [23]:
left = Daru::DataFrame.new(k: ['K0', 'K1', 'K2'], v: [1, 2, 3])
right = Daru::DataFrame.new(k: ['K0', 'K0', 'K3'], v: [4, 5, 6])

left.join(right, on: [:k], how: :inner)

#<Daru::DataFrame(2x3)>
     v_1   k v_2
   0   1  K0   4
   1   1  K0   5

In [24]:
## Can't specify suffixes

### Joining multiple DataFrame or Panel objects

In [25]:
## Can't join multiple DataFrame at once

### Merging Ordered Data

In [26]:
## I can't understand pandas's ordered_merge method's behavior.

### Merging together values within Series or DataFrame columns

In [27]:
## there's no methods like 'combine_first' or 'update'.

# lacking features

## create new multi-index when concatenating
like [pandas's "keys" option](http://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects)

## concatenation doens't keep multi-index
when concatenating two dataframes with multi-index, created dataframe doesn't keep (multi-)index.

## can't concat Vector with DataFrame directory

```Ruby
df.concat(vect) # fails
df.concat(vect.to_df) # successes 
```

## 